In [102]:
import requests
import random
import pandas as pd 
from tqdm import tqdm
from bs4  import BeautifulSoup
from cred_her import YOUR_API_KEY
import json 
from tqdm import tqdm
import time

import ast

In [103]:
YOUR_API_KEY = YOUR_API_KEY
YOUR_API_KEY

'gq4w-EXldLOsMbYDKBlcYx_h7mkjK0HBDrEyYX7pbnQ'

In [3]:
df = pd.read_csv('precios_de_combustibles.csv')

In [4]:
df.head(2)

,departamentocodigo,departamentonombre,municipiocodigo,municipionombre,agente,bandera,direccion,producto,precio,estado,fecharegistro
0,18,CAQUETA,18756,SOLANO,COMERCIALIZADORA EL HUECO,TERPEL,Puerto Principal,BIODIESEL EXTRA,8500,1,01/08/2015 12:00:00 AM
1,99,VICHADA,99624,SANTA ROSALIA,ESTACION DE SERVICIO LA VICTORIA,BRIO,Carrera 9 con calle 3 No. 08-92-96,BIODIESEL EXTRA,9000,1,01/08/2016 12:00:00 AM


<h1><center>Geocoding Api's</center></h1>

## Google API

* [**Google_Free_Tier**](https://cloud.google.com/free/?utm_source=google&utm_medium=cpc&utm_campaign=latam-LATAM-all-es-dr-AKWS-all-all-trial-p-dr-1011454-LUAC0013610&utm_content=text-ad-none-any-DEV_c-CRE_540885993580-ADGP_Hybrid%20%7C%20BKWS%20-%20PHR%20%7C%20Txt%20~%20Management-Tools_Console-KWID_43700065922071537-kwd-651816590614&utm_term=KW_google%20console-ST_Google%20Console&gclid=Cj0KCQjwnbmaBhD-ARIsAGTPcfWVMlJlTFywFNaAo8P-MNL4_gM6L7yucqAQ37xg1AIEHzpAzF_kkAoaAsszEALw_wcB&gclsrc=aw.ds)
* [**Google_Api_Pricing**](https://mapsplatform.google.com/pricing/)

## HERE Platform

* [**Here_Register**](https://platform.here.com/sign-up?step=verify-identity&aid=www.here.com-start-building)
* [**Here_Princing**](https://www.here.com/pricing)
* [**Geocoding_Documentation**](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics/endpoint-geocode-brief.html)

# Clean Data 

In [34]:
df["fecharegistro"] = pd.to_datetime(df["fecharegistro"])

# Extraer el año y guardar el resultado en la columna "periodo"
df["periodo"] = df["fecharegistro"].dt.year.apply(str).str[-2:]
df["periodo"] = "20" + df["periodo"]

# Convertir la columna "fecharegistro" a objetos de tipo fecha
df["fecharegistro"] = pd.to_datetime(df["fecharegistro"])

# Extraer el mes y guardar el resultado en la columna "mes"
df["mes"] = df["fecharegistro"].dt.month

In [36]:
df[df["periodo"] == "2018"]["mes"].max()

5

In [38]:
df

,departamentocodigo,departamentonombre,municipiocodigo,municipionombre,agente,bandera,direccion,producto,precio,estado,fecharegistro,periodo,mes
0,18,CAQUETA,18756,SOLANO,COMERCIALIZADORA EL HUECO,TERPEL,Puerto Principal,BIODIESEL EXTRA,8500,1,2015-01-08,2015,1
1,99,VICHADA,99624,SANTA ROSALIA,ESTACION DE SERVICIO LA VICTORIA,BRIO,Carrera 9 con calle 3 No. 08-92-96,BIODIESEL EXTRA,9000,1,2016-01-08,2016,1
2,25,CUNDINAMARCA,25175,CHIA,COMERCIAL SANTA INES LIMITADA EN LIQUIDACION,TERPEL,CARRERA 1 No. 6A-60,GASOLINA CORRIENTE OXIGENADA,8060,1,2017-01-04,2017,1
3,5,ANTIOQUIA,5837,TURBO,BOMBA CODI EL PUEBLO,PROXXON,CRA 13 99 A 41,BIODIESEL EXTRA,8893,1,2018-05-07,2018,5
4,47,MAGDALENA,47605,REMOLINO,ESTACION DE SERVICIOS LA RUTA,ECOS,FINCA EL CHORY ENTRADA A REMOLINO EN REMOLINO,BIODIESEL EXTRA,7500,1,2017-01-28,2017,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13569,50,META,50711,VISTAHERMOSA,ESTACION DE SERVICIO LOS TRILLIZOS,BIOMAX,CALLE PRINCIPAL VEREDA MARACAIBO,BIOACEM AL 9%,9000,1,2017-09-04,2017,9
13570,25,CUNDINAMARCA,25095,BITUIMA,ESTACION DE SERVICIO LA SIERRA,BIOMAX,IPM LA SIERRA CENTRO,GASOLINA CORRIENTE OXIGENADA,9214,1,2018-05-04,2018,5
13571,85,CASANARE,85001,YOPAL,ESTACION DE SERVICIO TRANSPORTADORES UNIDOS LTDA,PETROBRAS,Calle 25 No 22-40,GASOLINA CORRIENTE OXIGENADA,9249,1,2018-05-03,2018,5
13572,66,RISARALDA,66001,PEREIRA,ESTACION DE SERVICIO AUTOS GAMA,BRIO,CALLE 21 No. 12-41,GASOLINA EXTRA OXIGENADA,11290,1,2018-05-03,2018,5


In [47]:
df2 =  df[(df['periodo']=="2018") & (df['mes']==5)]
df2 =  df2[(df2['departamentonombre']=='ANTIOQUIA') | (df2['departamentonombre']=='BOGOTA D.C.')]

In [48]:
df2

,departamentocodigo,departamentonombre,municipiocodigo,municipionombre,agente,bandera,direccion,producto,precio,estado,fecharegistro,periodo,mes
3,5,ANTIOQUIA,5837,TURBO,BOMBA CODI EL PUEBLO,PROXXON,CRA 13 99 A 41,BIODIESEL EXTRA,8893,1,2018-05-07,2018,5
64,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,2018-05-02,2018,5
67,11,BOGOTA D.C.,11001,BOGOTA D.C.,EDS ESSO INDUSTRIAL BOYACA,MOBIL,CARRERA 71 No. 19-05,GASOLINA CORRIENTE OXIGENADA,9090,1,2018-05-02,2018,5
73,5,ANTIOQUIA,5642,SALGAR,SERVICENTRO ZEUSS SALGAR,ZEUSS,SECTOR EL PUENTE RESTREPO VEREDA BELLAVISTA,BIODIESEL EXTRA,8767,1,2018-05-04,2018,5
78,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO AUTOMOTRIZ PETROBRAS CALL...,PETROBRAS,CALLE 45 No. 16-30,GASOLINA CORRIENTE OXIGENADA,9230,1,2018-05-01,2018,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13554,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIOS ESSO LA ESTRELLA,MOBIL,CARRERA 129 No. 132A-88,BIODIESEL EXTRA,8448,1,2018-05-03,2018,5
13562,5,ANTIOQUIA,5591,PUERTO TRIUNFO,ESTACIÓN DE SERVICIOS NAPOLES,BIOMAX,KILOMETRO 120 MAS 400 AUTOPISTA MEDELLÍN - BOGOTÁ,BIODIESEL EXTRA,8000,1,2018-05-04,2018,5
13563,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO SAN ANTONIO,MOBIL,CARRERA 16 No. 183 - 44,GASOLINA CORRIENTE OXIGENADA,8893,1,2018-05-02,2018,5
13566,11,BOGOTA D.C.,11001,BOGOTA D.C.,EDS LOS ALPES,BIOMAX,CALLE 36 F BIS SUR No. 9 - 35 ESTE,GASOLINA CORRIENTE OXIGENADA,8815,1,2018-05-02,2018,5


In [49]:
df2['municipionombre'].value_counts().head(10)

BOGOTA D.C.           1278
MEDELLIN               305
BELLO                   59
ITAGUI                  53
CANON DE RIO CLARO      48
ENVIGADO                46
CAUCASIA                28
SABANETA                27
TURBO                   25
COPACABANA              25
Name: municipionombre, dtype: int64

In [61]:
# Crear una lista de municipios para filtrar
municipios = ['BOGOTA D.C.', 'MEDELLIN', 'BELLO', 'ITAGUI', 'CANON DE RIO CLARO', 'ENVIGADO', 'CAUCASIA']

# Seleccionar las filas correspondientes a "BOGOTA D.C." y "ANTIOQUIA" con los municipios en la lista
df3 = df2.loc[(df2['departamentonombre'].isin(['BOGOTA D.C.', 'ANTIOQUIA'])) & (df2['municipionombre'].isin(municipios))]
df3.reset_index(inplace = True)
df3.drop(columns = 'index',inplace = True)

C:\Users\Jiord\AppData\Local\Temp\ipykernel_24440\3462159036.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.drop(columns = 'index',inplace = True)


In [62]:
df3 = df3.rename(columns={"municipionombre": "Municipio", "departamentonombre": "Departamento", "direccion":"Direccion", "agente":"Nombre_comercial"})
df3.head(2)

,departamentocodigo,Departamento,municipiocodigo,Municipio,Nombre_comercial,bandera,Direccion,producto,precio,estado,fecharegistro,periodo,mes
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,2018-05-02,2018,5
1,11,BOGOTA D.C.,11001,BOGOTA D.C.,EDS ESSO INDUSTRIAL BOYACA,MOBIL,CARRERA 71 No. 19-05,GASOLINA CORRIENTE OXIGENADA,9090,1,2018-05-02,2018,5


In [63]:
df3['Municipio'].unique()

array(['BOGOTA D.C.', 'MEDELLIN', 'ENVIGADO', 'CANON DE RIO CLARO',
       'ITAGUI', 'CAUCASIA', 'BELLO'], dtype=object)

In [64]:
df3['Dir2'] = df3['Direccion'].apply(lambda x : x.replace('#','No'))
df3['Full_Address'] = df3.apply(lambda x : x['Direccion'] + ', ' + x['Municipio'].capitalize() + ', Colombia' , axis = 1)

In [65]:
df3['Full_Address'].nunique()

577

In [66]:
df3['Full_Address'][0]

'CARRERA 120 No. 17-37, Bogota d.c., Colombia'

In [67]:
df3

,departamentocodigo,Departamento,municipiocodigo,Municipio,Nombre_comercial,bandera,Direccion,producto,precio,estado,fecharegistro,periodo,mes,Dir2,Full_Address
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,2018-05-02,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia"
1,11,BOGOTA D.C.,11001,BOGOTA D.C.,EDS ESSO INDUSTRIAL BOYACA,MOBIL,CARRERA 71 No. 19-05,GASOLINA CORRIENTE OXIGENADA,9090,1,2018-05-02,2018,5,CARRERA 71 No. 19-05,"CARRERA 71 No. 19-05, Bogota d.c., Colombia"
2,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO AUTOMOTRIZ PETROBRAS CALL...,PETROBRAS,CALLE 45 No. 16-30,GASOLINA CORRIENTE OXIGENADA,9230,1,2018-05-01,2018,5,CALLE 45 No. 16-30,"CALLE 45 No. 16-30, Bogota d.c., Colombia"
3,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL CAMPO VALDES,TERPEL,CARRERA 49 No. 84-61,GASOLINA EXTRA OXIGENADA,11980,1,2018-05-03,2018,5,CARRERA 49 No. 84-61,"CARRERA 49 No. 84-61, Medellin, Colombia"
4,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO SERVICENTRO ESSO EL SOCIEGO,MOBIL,Carrera 10 No. 27-05 Sur,BIODIESEL EXTRA,8450,1,2018-05-02,2018,5,Carrera 10 No. 27-05 Sur,"Carrera 10 No. 27-05 Sur, Bogota d.c., Colombia"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO DISTRACOM PEDREGAL,MOBIL,Carrera 64 C No.89 A-30,BIODIESEL EXTRA,8420,1,2018-05-03,2018,5,Carrera 64 C No.89 A-30,"Carrera 64 C No.89 A-30, Medellin, Colombia"
1813,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIOS ESSO LA ESTRELLA,MOBIL,CARRERA 129 No. 132A-88,BIODIESEL EXTRA,8448,1,2018-05-03,2018,5,CARRERA 129 No. 132A-88,"CARRERA 129 No. 132A-88, Bogota d.c., Colombia"
1814,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO SAN ANTONIO,MOBIL,CARRERA 16 No. 183 - 44,GASOLINA CORRIENTE OXIGENADA,8893,1,2018-05-02,2018,5,CARRERA 16 No. 183 - 44,"CARRERA 16 No. 183 - 44, Bogota d.c., Colombia"
1815,11,BOGOTA D.C.,11001,BOGOTA D.C.,EDS LOS ALPES,BIOMAX,CALLE 36 F BIS SUR No. 9 - 35 ESTE,GASOLINA CORRIENTE OXIGENADA,8815,1,2018-05-02,2018,5,CALLE 36 F BIS SUR No. 9 - 35 ESTE,"CALLE 36 F BIS SUR No. 9 - 35 ESTE, Bogota d.c..."


In [68]:
estaciones =  df3[['Nombre_comercial','Direccion','Full_Address']].drop_duplicates(subset = 'Nombre_comercial')
estaciones.reset_index(inplace = True)
estaciones.drop(columns = 'index',inplace = True)

In [69]:
estaciones['Full_Address'][0]

'CARRERA 120 No. 17-37, Bogota d.c., Colombia'

In [70]:
estaciones

,Nombre_comercial,Direccion,Full_Address
0,ESTACION DE SERVICIO METROPOLITANA,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia"
1,EDS ESSO INDUSTRIAL BOYACA,CARRERA 71 No. 19-05,"CARRERA 71 No. 19-05, Bogota d.c., Colombia"
2,ESTACION DE SERVICIO AUTOMOTRIZ PETROBRAS CALL...,CALLE 45 No. 16-30,"CALLE 45 No. 16-30, Bogota d.c., Colombia"
3,ESTACION DE SERVICIO TERPEL CAMPO VALDES,CARRERA 49 No. 84-61,"CARRERA 49 No. 84-61, Medellin, Colombia"
4,ESTACION DE SERVICIO SERVICENTRO ESSO EL SOCIEGO,Carrera 10 No. 27-05 Sur,"Carrera 10 No. 27-05 Sur, Bogota d.c., Colombia"
...,...,...,...
570,ESTACION DE SERVICIOS DISTRACOM EXPOSICIONES,CALLE 36 No. 51-18,"CALLE 36 No. 51-18, Medellin, Colombia"
571,ESTACION DE SERVICIO JARDIN,Carrera 51 No.73-20 San Pedro,"Carrera 51 No.73-20 San Pedro, Medellin, Colombia"
572,MOBIL COLMOTORES,TRANSVERSAL 60 No. 51A-08 22 24 SUR,"TRANSVERSAL 60 No. 51A-08 22 24 SUR, Bogota d...."
573,ESTACION DE SERVICIO TERMINAL,CALLE 62 No. 131-80 Corregimiento San Cristobal,CALLE 62 No. 131-80 Corregimiento San Cristoba...


In [104]:
address = estaciones['Full_Address'][0]
url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
response = requests.get(url).json()

In [105]:
response

{'items': [{'title': 'Carrera 120 17-37, 110921 Bogotá, D.C., Colombia',
   'id': 'here:af:streetsection:6pAtCjR-qQKDAwmYp8TiFC:CggIBCC88KeqAhABGgIzNw',
   'resultType': 'houseNumber',
   'houseNumberType': 'PA',
   'address': {'label': 'Carrera 120 17-37, 110921 Bogotá, D.C., Colombia',
    'countryCode': 'COL',
    'countryName': 'Colombia',
    'countyCode': 'DC',
    'county': 'Bogotá, D.C.',
    'city': 'Bogotá, D.C.',
    'district': 'Fontibón',
    'street': 'Carrera 120 17',
    'postalCode': '110921',
    'houseNumber': '37'},
   'position': {'lat': 4.68585, 'lng': -74.15736},
   'access': [{'lat': 4.68579, 'lng': -74.15731}],
   'mapView': {'west': -74.15826,
    'south': 4.68495,
    'east': -74.15646,
    'north': 4.68675},
   'scoring': {'queryScore': 0.92,
    'fieldScore': {'country': 1.0,
     'city': 1.0,
     'streets': [0.96],
     'houseNumber': 1.0}}},
  {'title': 'Carrera 120 17F-37, 110921 Bogotá, D.C., Colombia',
   'id': 'here:af:streetsection:7nxxa6p-sWYzk3AQK

In [106]:
def Get_Coords(address,YOUR_API_KEY):
    
    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
    
    try : 
        
        response = requests.get(url).json()
        CleanAddress = response['items'][0]['title'].upper()
        LAT = response['items'][0]['position']['lat']
        LNG = response['items'][0]['position']['lng']
        
        results = [CleanAddress,LAT,LNG]
        
    except :
        
        results = ['Not Found','NA','NA']
        
    
    return results 

In [107]:
Get_Coords(address,YOUR_API_KEY)

['CARRERA 120 17-37, 110921 BOGOTÁ, D.C., COLOMBIA', 4.68585, -74.15736]

# Extracción de coordenadas 

In [108]:
coordenadas = []
tiempo = [1,2,2.5]

for i in tqdm(range(len(estaciones['Full_Address'])),colour = 'green'):
    
    try :
        
        R = Get_Coords(estaciones['Full_Address'][i],YOUR_API_KEY)
        geo_source = R[1],R[2]
        coordenadas.append(geo_source)
        
    except :
        
        coordenadas.append('Error')
        
    
    time.sleep(random.choices(tiempo)[0])
    
        

100%|██████████| 575/575 [23:05<00:00,  2.41s/it]


In [109]:
estaciones['Coords'] = coordenadas

In [110]:
len(df3)

1817

In [111]:
estaciones.head(2)

,Nombre_comercial,Direccion,Full_Address,Coords
0,ESTACION DE SERVICIO METROPOLITANA,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(4.68585, -74.15736)"
1,EDS ESSO INDUSTRIAL BOYACA,CARRERA 71 No. 19-05,"CARRERA 71 No. 19-05, Bogota d.c., Colombia","(4.65127, -74.1234)"


In [112]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [119]:
df_final.head(3)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords
0,2022,6,5,ANTIOQUIA,5266,ENVIGADO,ESTACION DE SERVICIOS PREMIUM LA SEBASTIANA,TEXACO,CARRERA 27 No. 36 SUR-199,GASOLINA CORRIENTE OXIGENADA,9710.0,A,CARRERA 27 No. 36 SUR-199,"CARRERA 27 No. 36 SUR-199, Envigado, Colombia","(6.16364, -75.56929)"
1,2022,6,5,ANTIOQUIA,5266,ENVIGADO,ESTACION DE SERVICIOS PREMIUM LA SEBASTIANA,TEXACO,CARRERA 27 No. 36 SUR-199,BIODIESEL EXTRA,10510.0,A,CARRERA 27 No. 36 SUR-199,"CARRERA 27 No. 36 SUR-199, Envigado, Colombia","(6.16364, -75.56929)"
2,2022,6,5,ANTIOQUIA,5266,ENVIGADO,ESTACION DE SERVICIOS PREMIUM LA SEBASTIANA,TEXACO,CARRERA 27 No. 36 SUR-199,BIODIESEL EXTRA,10510.0,A,CARRERA 27 No. 36 SUR-199,"CARRERA 27 No. 36 SUR-199, Envigado, Colombia","(6.16364, -75.56929)"


In [113]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [114]:
df_final.to_csv('DATASET_FINAL.CSV',index = False)

In [115]:
df_final

,departamentocodigo,Departamento,municipiocodigo,Municipio,Nombre_comercial,bandera,Direccion,producto,precio,estado,fecharegistro,periodo,mes,Dir2,Full_Address,Coords
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,2018-05-02,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(4.68585, -74.15736)"
1,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,GASOLINA CORRIENTE OXIGENADA,9095,1,2018-05-02,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(4.68585, -74.15736)"
2,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIOACEM AL 9%,8400,1,2018-05-02,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(4.68585, -74.15736)"
3,11,BOGOTA D.C.,11001,BOGOTA D.C.,EDS ESSO INDUSTRIAL BOYACA,MOBIL,CARRERA 71 No. 19-05,GASOLINA CORRIENTE OXIGENADA,9090,1,2018-05-02,2018,5,CARRERA 71 No. 19-05,"CARRERA 71 No. 19-05, Bogota d.c., Colombia","(4.65127, -74.1234)"
4,11,BOGOTA D.C.,11001,BOGOTA D.C.,EDS ESSO INDUSTRIAL BOYACA,MOBIL,CARRERA 71 No. 19-05,BIODIESEL EXTRA,9190,1,2018-05-02,2018,5,CARRERA 71 No. 19-05,"CARRERA 71 No. 19-05, Bogota d.c., Colombia","(4.65127, -74.1234)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,11,BOGOTA D.C.,11001,BOGOTA D.C.,MOBIL COLMOTORES,MOBIL,TRANSVERSAL 60 No. 51A-08 22 24 SUR,BIOACEM AL 9%,7909,1,2018-05-02,2018,5,TRANSVERSAL 60 No. 51A-08 22 24 SUR,"TRANSVERSAL 60 No. 51A-08 22 24 SUR, Bogota d....","(4.58994, -74.14506)"
1809,11,BOGOTA D.C.,11001,BOGOTA D.C.,MOBIL COLMOTORES,MOBIL,TRANSVERSAL 60 No. 51A-08 22 24 SUR,GASOLINA CORRIENTE OXIGENADA,8499,1,2018-05-02,2018,5,TRANSVERSAL 60 No. 51A-08 22 24 SUR,"TRANSVERSAL 60 No. 51A-08 22 24 SUR, Bogota d....","(4.58994, -74.14506)"
1810,11,BOGOTA D.C.,11001,BOGOTA D.C.,MOBIL COLMOTORES,MOBIL,TRANSVERSAL 60 No. 51A-08 22 24 SUR,BIODIESEL EXTRA,7909,1,2018-05-02,2018,5,TRANSVERSAL 60 No. 51A-08 22 24 SUR,"TRANSVERSAL 60 No. 51A-08 22 24 SUR, Bogota d....","(4.58994, -74.14506)"
1811,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERMINAL,MOBIL,CALLE 62 No. 131-80 Corregimiento San Cristobal,BIODIESEL EXTRA,8382,1,2018-05-04,2018,5,CALLE 62 No. 131-80 Corregimiento San Cristobal,CALLE 62 No. 131-80 Corregimiento San Cristoba...,"(6.27818, -75.63433)"
